In [4]:
# import serial
import time
import ezdxf as edf
import math
import pickle
import time

from datetime import datetime

from helpers.lines import *
from helpers.curves import *
from helpers.layers import *


In [2]:

doc = edf.readfile('/home/pi/cadpos/resources/dxfs/Drawing12 - office table test.dxf')
msp = doc.modelspace()
entities = getEntitiesInLayer(msp)
obstacles = entities['0']
WA = getWorkingArea(list(msp.query('LINE')))

methods = {
    'LWPOLYLINE' : is_point_in_polyline,
    'CIRCLE' : is_point_in_circ,
    'ELLIPSE' : is_point_in_ellipse,
    'SPLINE' : isInSPLine
}

discretisedWA = discretizeWorkingArea(WA)
obstaclePoints = []
counter =0
# for point in discretisedWA:
#     for entity in obstacles:
#         if entity.dxftype() in methods.keys():
#             labelPoint = methods[entity.dxftype()](point, entity)
#             if labelPoint >= 0:
#                 obstaclePoints.append((round(point[0],2),round(point[1],2)))
#             counter += 1
#             if counter %100000==0:
#                 print(counter)




100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000


In [5]:
# obstaclePoints = list(set(obstaclePoints))
with open('./resources/obsPoints','rb') as obsPoints_file:
    obstaclePoints = pickle.load(obsPoints_file)

In [11]:
sorted(obstaclePoints)[300000]

(25.99, 13.02)

In [21]:
def defineTagWorkingArea(orderOfTags, dxfWorkingArea):
    anchorXY = dict()
#     dxfWorkingArea = sorted(dxfWorkingArea, key=itemgetter(0,1))
    
    for i in range(len(orderOfTags)):
        anchorXY.update({orderOfTags[i]: dxfWorkingArea[i]})
    
    return anchorXY

def parseDeca(decaData, anchorXY):
    baseX = anchorXY['A'][0]
    baseY = anchorXY['A'][1]
    
    tagX = decaData['TAG'][0]
    tagY = decaData['TAG'][1]
    
    tagDXFX = baseX + tagX
    tagDXFY = baseY + tagY
    
    tagPosition = edf.math.Vec2(round(tagDXFX,2), round(tagDXFY,2))
    
    return tagPosition

In [22]:
DWM = serial.Serial(port="/dev/ttyACM0", baudrate=115200)
DWM.write("\r\r".encode())
print("Encode")
time.sleep(1)
DWM.write("lec\r".encode())

loc = {}

anchorXY = defineTagWorkingArea(['C','B','A'], WA)
with open('pos_log', 'w') as logfile:
    while True:
        time.sleep(1)
        data = DWM.readline().decode("utf-8") 
        fields = data.strip().split(',')
        if len(fields) < 6:
            continue
        print(data)
        for idx, field in enumerate(fields):
            if field == 'AN1':
                loc['A'] = [float(i) for i in fields[idx+2:idx+6]]
            elif field == 'AN2':
                loc['B'] =  [float(i) for i in  fields[idx+2:idx+6]]
            elif field == 'AN0':
                loc['C'] =  [float(i) for i in  fields[idx+2:idx+6]]
            elif field == 'POS':
                loc['TAG'] =  [float(i) for i in fields[idx+1:idx+4]]
        
        my_loc_in_dxf = parseDeca(loc, anchorXY)
        msg = f'{datetime.now()}: loc deca:{loc["TAG"]};  log dxf = {my_loc_in_dxf}\n'
#         logfile.write(msg)
        print(msg)
        if my_loc_in_dxf in obsta
#         if 
    

Encode
0,2.51,AN1,DC11,0.00,0.00,0.00,3.65,AN2,CCAC,1.20,0.00,0.00,3.64,POS,0.60,3.18,1.58,96

1603009221.8564446: loc deca:[0.6, 3.18, 1.58] ;  log dxf = (23.45, 21.28)

DIST,3,AN0,D206,1.20,1.35,0.00,2.46,AN1,DC11,0.00,0.00,0.00,3.65,AN2,CCAC,1.20,0.00,0.00,3.62,POS,0.62,3.21,1.55,97

1603009222.8613658: loc deca:[0.62, 3.21, 1.55] ;  log dxf = (23.47, 21.31)

DIST,3,AN0,D206,1.20,1.35,0.00,2.44,AN1,DC11,0.00,0

1603009223.8641028: loc deca:[0.62, 3.21, 1.55] ;  log dxf = (23.47, 21.31)

dwm> DIST,3,AN0,D206,1.20,1.35,0.00,2.59,AN1,DC11,0.00,0.00,0.00,3.37,AN2,CCAC,1.20,0.00,0.00,3.54,POS,0.08,2.98,1.63,94

1603009225.8745883: loc deca:[0.08, 2.98, 1.63] ;  log dxf = (22.93, 21.08)

DIST,3,AN0,D206,1.20,1.35,0.00,2.60,AN1,DC11,0.00,0.00,0.00,3.30,AN2,CCAC,1.20,0.00,0.00,3.50,POS,0.06,2.92,1.69,94

1603009226.8790207: loc deca:[0.06, 2.92, 1.69] ;  log dxf = (22.91, 21.02)

DIST,3,AN0,D206,1.20,1.35,0.00,2.60,AN1,DC11,0.00,0.00,0.00,3.29,AN2,CCAC,1.20,0.00,0.00,3.51,POS,0.05,2.87,1.72

KeyboardInterrupt: 

In [4]:
bah = b'dwm> DIST,3,AN0,D206,1.16,2.66,0.00,2.46,AN1,DC11,0.00,0.00,0.00,2.66,AN2,CCAC,1.82,0.00,0.00,3.70,POS,-0.82,2.21,1.34,88\r\n'

In [5]:
str(bah)

"b'dwm> DIST,3,AN0,D206,1.16,2.66,0.00,2.46,AN1,DC11,0.00,0.00,0.00,2.66,AN2,CCAC,1.82,0.00,0.00,3.70,POS,-0.82,2.21,1.34,88\\r\\n'"

In [6]:

bah.strip()

b'dwm> DIST,3,AN0,D206,1.16,2.66,0.00,2.46,AN1,DC11,0.00,0.00,0.00,2.66,AN2,CCAC,1.82,0.00,0.00,3.70,POS,-0.82,2.21,1.34,88'

In [ ]:
ba